**Table of contents**<a id='toc0_'></a>    
- 1. [Problem 1: Production economy and CO2 taxation](#toc1_)    
- 2. [Problem 2: Career choice model](#toc2_)    
- 3. [Problem 3: Barycentric interpolation](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [64]:
# Write your code here
import numpy as np
import pandas as pd

from scipy import optimize

np.random.seed(404)

## 1. <a id='toc1_'></a>[Problem 1: Production economy and CO2 taxation](#toc0_)

Consider a production economy with two firms indexed by $j \in \{1,2\}$. Each produce its own good. They solve

$$
\begin{align*}
\max_{y_{j}}\pi_{j}&=p_{j}y_{j}-w_{j}\ell_{j}\\\text{s.t.}\;&y_{j}=A\ell_{j}^{\gamma}.
\end{align*}
$$

Optimal firm behavior is

$$
\begin{align*}
\ell_{j}^{\star}(w,p_{j})&=\left(\frac{p_{j}A\gamma}{w}\right)^{\frac{1}{1-\gamma}} \\
y_{j}^{\star}(w,p_{j})&=A\left(\ell_{j}^{\star}(w,p_{j})\right)^{\gamma}
\end{align*}
$$

The implied profits are

$$
\pi_{j}^*(w,p_{j})=\frac{1-\gamma}{\gamma}w\cdot\left(\frac{p_{j}A\gamma}{w}\right)^{\frac{1}{1-\gamma}}
$$

A single consumer supplies labor, and consumes the goods the firms produce. She also recieves the implied profits of the firm.<br>
She solves:

$$
\begin{align*}
U(p_1,p_2,w,\tau,T) = \max_{c_{1},c_{2},\ell} & \log(c_{1}^{\alpha}c_{2}^{1-\alpha})-\nu\frac{\ell^{1+\epsilon}}{1+\epsilon} \\
\text{s.t.}\,\,\,&p_{1}c_{1}+(p_{2}+\tau)c_{2}=w\ell+T+\pi_{1}^*(w,p_{1})+\pi_{2}^*(w,p_{2})
\end{align*}
$$

where $\tau$ is a tax and $T$ is lump-sum transfer. <br>
For a given $\ell$, it can be shown that optimal behavior is

$$
\begin{align*}
c_{1}(\ell)&=\alpha\frac{w\ell+T+\pi_{1}^*(w,p_{1})+\pi_{2}^*(w,p_{2})}{p_{1}} \\
c_{2}(\ell)&=(1-\alpha)\frac{w\ell+T+\pi_{1}^*(w,p_{1})+\pi_{2}^*(w,p_{2})}{p_{2}+\tau} \\
\end{align*}
$$
Such that optimal behavior is:
$$
\ell^* = \underset{\ell}{\arg\max} \log(\left(c_{1}(\ell)\right)^{\alpha}\cdot \left(c_{2}(\ell)\right)^{1-\alpha})-\nu\frac{\ell^{1+\epsilon}}{1+\epsilon} 
$$
With optimal consumption:
$$
\begin{align*}
c_1^*=c_{1}(\ell^*) \\
c_2^*=c_{2}(\ell^*)\\
\end{align*}
$$


The government chooses $\tau$ and balances its budget so $T=\tau c_2^*$. We initially set $\tau,T=0$.

Market clearing requires:

1. Labor market: $\ell^* = \ell_1^* + \ell_2^*$
1. Good market 1: $c_1^* = y_1^*$
1. Good market 2: $c_2^* = y_2^*$


**Question 1:** Check market clearing conditions for $p_1$ in `linspace(0.1,2.0,10)` and $p_2$ in `linspace(0.1,2.0,10)`. We choose $w=1$ as numeraire.

In [2]:
# firms
A = 1.0
gamma = 0.5

# households
alpha = 0.3
nu = 1.0
epsilon = 2.0

# government
tau = 0.0
T = 0.0

# Question 3
kappa = 0.1

In [3]:
def l_star(p):
    return (p*A*gamma)**(1/(1-gamma))
def y_star(p):
    return A*(l_star(p))**(gamma)
def pi_star(p):
    return ((1-gamma)/(gamma))*(l_star(p))

def utility(c1, c2, L):
    return np.log(c1**alpha*c2**(1-alpha))-nu*(L)**(1+epsilon)/(1+epsilon)

# Consumer's optimal behavior for a given labor supply
def c1_star(L, p1, p2, T):        
        return alpha * (L + T + pi_star(p1) + pi_star(p2)) / p1

def c2_star(L, p1, p2, T, tau):
        return (1 - alpha) * (L + T + pi_star(p1) + pi_star(p2)) / (p2 + tau)

def obj(L, p1, p2, T, tau):
      return np.log((c1_star(L, p1, p2, T))**(alpha) * (c2_star(L, p1, p2, T, tau))**(1-alpha)) - nu * (L**(1+epsilon))/(1+epsilon)

In [4]:
P = (np.linspace(0.1, 2, 10), np.linspace(0.1, 2, 10))
P1 = np.linspace(0.1, 2, 10)
P2 = np.linspace(0.1, 2, 10)

res = []

for p1 in P1:
    for p2 in P2:
        l_star1 = l_star(p1)
        l_star2 = l_star(p2)
        y_star1 = y_star(p1)
        y_star2 = y_star(p2) 

        pi_star1 = pi_star(p1)
        pi_star2 = pi_star(p2)

        res_opt = optimize.minimize(lambda L: -obj(L, p1, p2, T, tau), x0=0, method='BFGS')

        L_star = res_opt.x[0]
        u_star = res_opt.fun

        # Calculate optimal consumption
        c1_star_opt = c1_star(L_star, p1, p2, T)
        c2_star_opt = c2_star(L_star, p1, p2, T, tau)

        # Market clears?
        cond1 = np.isclose(L_star, l_star1 + l_star2)
        cond2 = np.isclose(c1_star_opt, y_star1)
        cond3 = np.isclose(c2_star_opt, y_star2)

        excess_good1 = abs(c1_star_opt - y_star1)
        excess_good2 = abs(c2_star_opt - y_star2)
        excess_labor = abs(L_star - l_star1 - l_star2)

        res.append({
            'p1': p1.round(2),
            'p2': p2.round(2),
            'goods market clearing': (cond2, cond3),
            'utility': u_star,
            'labor': L_star,
            'labor market clearing': cond1,
            'excess goods': (excess_good1.round(2), excess_good2.round(2)),
            'excess labor': excess_labor.round(2)
        })

import pandas as pd

tab1 = pd.DataFrame(res)
tab1

,p1,p2,goods market clearing,utility,labor,labor market clearing,excess goods,excess labor
0,0.1,0.10,"(False, False)",-1.363379,0.998337,False,"(2.96, 6.97)",0.99
1,0.1,0.31,"(False, False)",-0.590364,0.991180,False,"(3.0, 2.13)",0.96
2,0.1,0.52,"(False, False)",-0.270394,0.976987,False,"(3.09, 1.14)",0.91
3,0.1,0.73,"(False, False)",-0.094654,0.956373,False,"(3.23, 0.68)",0.82
4,0.1,0.94,"(False, False)",0.003669,0.930205,False,"(3.42, 0.38)",0.70
...,...,...,...,...,...,...,...,...
95,2.0,1.16,"(False, False)",0.324428,0.701025,False,"(0.69, 0.65)",0.63
96,2.0,1.37,"(False, False)",0.378232,0.682140,False,"(0.68, 0.42)",0.78
97,2.0,1.58,"(False, False)",0.408647,0.661682,False,"(0.66, 0.22)",0.96
98,2.0,1.79,"(False, False)",0.421295,0.640165,False,"(0.63, 0.06)",1.16


In [5]:
idx_goods = tab1['excess goods'].min()
tab1[tab1['excess goods']==idx_goods]

,p1,p2,goods market clearing,utility,labor,labor market clearing,excess goods,excess labor
46,0.94,1.37,"(False, False)",0.584075,0.815123,False,"(0.01, 0.09)",0.13


**Question 2:** Find the equilibrium prices $p_1$ and $p_2$.<br>
*Hint: you can use Walras' law to only check 2 of the market clearings*

In [6]:
# For which p1, p2 has the lowest market clearing error?
idx_goods = tab1['excess goods'].min()
p1_star, p2_star = tab1[tab1['excess goods']==idx_goods].loc[:, ['p1', 'p2']].values[0]
L_star_opt = tab1[tab1['excess goods']==idx_goods].loc[:, ['labor']].values[0]

print(f'Prices (p1, p2)=({p1_star:.2f}, {p2_star:.2f}) has the lowest market clearing error and is thus the equilibrium prices.')

Prices (p1, p2)=(0.94, 1.37) has the lowest market clearing error and is thus the equilibrium prices.


Assume the government care about the social welfare function:

$$
SWF = U - \kappa y_2^*
$$

Here $\kappa$ measures the social cost of carbon emitted by the production of $y_2$ in equilibrium.

**Question 3:** What values of $\tau$ and (implied) $T$ should the government choose to maximize $SWF$?

In [7]:
def obj_gov(L, p1, p2, T, tau):
      return np.log((c1_star(L, p1, p2, T))**(alpha) * (c2_star(L, p1, p2, T, tau))**(1-alpha)) - nu * (L**(1+epsilon))/(1+epsilon) - kappa*y_star(p2)

In [8]:
y_star2_opt = y_star(p2_star)

In [ ]:
def obj(L, p1, p2, T, tau):
      return np.log((c1_star(L, p1, p2, T))**(alpha) * (c2_star(L, p1, p2, T, tau))**(1-alpha)) - nu * (L**(1+epsilon))/(1+epsilon)

In [19]:
tau
T

0.0

In [21]:
tau = 0.5
T = tau*c2_star_opt
for p1 in P1:
    for p2 in P2:
        l_star1 = l_star(p1)
        l_star2 = l_star(p2)
        y_star1 = y_star(p1)
        y_star2 = y_star(p2) 

        pi_star1 = pi_star(p1)
        pi_star2 = pi_star(p2)

        res_opt = optimize.minimize(lambda L: -obj(L, p1, p2, T, tau), x0=0, method='BFGS')

        L_star = res_opt.x[0]
        u_star = res_opt.fun

        # Calculate optimal consumption
        c1_star_opt = c1_star(L_star, p1, p2, T)
        c2_star_opt = c2_star(L_star, p1, p2, T, tau)

        # Market clears?
        cond1 = np.isclose(L_star, l_star1 + l_star2)
        cond2 = np.isclose(c1_star_opt, y_star1)
        cond3 = np.isclose(c2_star_opt, y_star2)

        excess_good1 = abs(c1_star_opt - y_star1)
        excess_good2 = abs(c2_star_opt - y_star2)
        excess_labor = abs(L_star - l_star1 - l_star2)

        res.append({
            'p1': p1.round(2),
            'p2': p2.round(2),
            'goods market clearing': (cond2, cond3),
            'utility': u_star,
            'labor': L_star,
            'labor market clearing': cond1,
            'excess goods': (excess_good1.round(2), excess_good2.round(2)),
            'excess labor': excess_labor.round(2)
        })

import pandas as pd

tab2 = pd.DataFrame(res)
tab2

,p1,p2,goods market clearing,utility,labor,labor market clearing,excess goods,excess labor
0,0.1,0.10,"(False, False)",-1.363379,0.998337,False,"(2.96, 6.97)",0.99
1,0.1,0.31,"(False, False)",-0.590364,0.991180,False,"(3.0, 2.13)",0.96
2,0.1,0.52,"(False, False)",-0.270394,0.976987,False,"(3.09, 1.14)",0.91
3,0.1,0.73,"(False, False)",-0.094654,0.956373,False,"(3.23, 0.68)",0.82
4,0.1,0.94,"(False, False)",0.003669,0.930205,False,"(3.42, 0.38)",0.70
...,...,...,...,...,...,...,...,...
295,2.0,1.16,"(False, False)",0.408678,0.652002,False,"(0.65, 0.42)",0.68
296,2.0,1.37,"(False, False)",0.437467,0.636313,False,"(0.63, 0.24)",0.83
297,2.0,1.58,"(False, False)",0.451243,0.619209,False,"(0.61, 0.09)",1.00
298,2.0,1.79,"(False, False)",0.452851,0.601097,False,"(0.58, 0.05)",1.20


In [22]:
obj(L_star_opt, p1_star, p2_star, 0, 0)

array([-0.58424103])

In [12]:
c1_star_opt

0.3927045365570219

In [13]:
c2_star_opt

0.9163105852997179

In [15]:
SWF = utility(c1_star_opt, c2_star_opt, L_star_opt) - kappa*y_star2_opt
SWF

array([-0.59061851])

In [17]:
def SWF(c1_star_opt, c2_star_opt, L_star_opt, tau):

    T = tau*c2_star_opt

    c1 = c1_star(L_star_opt, p1,  p2, T)
    c2 = c2_star(L_star_opt, p1, p2, T, tau)
    return -(np.log(c1**alpha*c2**(1-alpha))-nu*(L_star_opt)**(1+epsilon)/(1+epsilon) - kappa*y_star2_opt)

In [ ]:
SWF()

In [142]:
res = optimize.minimize_scalar(obj, method='bounded', bounds=(0, 10))
res.x

array([5.36344551e-06])

## 2. <a id='toc2_'></a>[Problem 2: Career choice model](#toc0_)

Consider a graduate $i$ making a choice between entering $J$ different career tracks. <br>
Entering career $j$ yields utility $u^k_{ij}$. This value is unknown to the graduate ex ante, but will ex post be: <br>
$$
    u_{i,j}^k = v_{j} + \epsilon_{i,j}^k
$$

They know that $\epsilon^k_{i,j}\sim \mathcal{N}(0,\sigma^2)$, but they do not observe $\epsilon^k_{i,j}$ before making their career choice. <br>

Consider the concrete case of $J=3$ with:
$$
\begin{align*}
    v_{1} &= 1 \\
    v_{2} &= 2 \\
    v_{3} &= 3
\end{align*}
$$

If the graduates know the values of $v_j$ and the distribution of $\epsilon_{i,j}^k$, they can calculate the expected utility of each career track using simulation: <br>
$$
    \mathbb{E}\left[ u^k_{i,j}\vert v_j \right] \approx v_j + \frac{1}{K}\sum_{k=1}^K \epsilon_{i,j}^k
$$

In [23]:
J = 3
N = 10
K = 10000

F = np.arange(1,N+1)
sigma = 2

v = np.array([1,2,3])
c = 1

**Question 1:** Simulate and calculate expected utility and the average realised utility for $K=10000$ draws, for each career choice $j$.


In [74]:
eps_ijk = [np.random.normal(0, sigma, size=K), np.random.normal(0, sigma, size=K), np.random.normal(0, sigma, size=K)]

In [75]:
res = []

res.append({
    'u1': v[0] + 1/K * eps_ijk[0].sum(),
    'u2': v[1] + 1/K * eps_ijk[1].sum(),
    'u3': v[2] + 1/K * eps_ijk[2].sum()
})

tab3 = pd.DataFrame(res)
tab3

,u1,u2,u3
0,1.042342,2.002273,2.984203


In [76]:
res = []

res.append({
    'u1': v[0] + np.mean(eps_ijk[0]),
    'u2': v[1] + np.mean(eps_ijk[1]),
    'u3': v[2] + np.mean(eps_ijk[2])
})

tab3 = pd.DataFrame(res)
tab3

,u1,u2,u3
0,1.042342,2.002273,2.984203


Now consider a new scenario: Imagine that the graduate does not know $v_j$. The *only* prior information they have on the value of each job, comes from their $F_{i}$ friends that work in each career $j$. After talking with them, they know the average utility of their friends (which includes their friends' noise term), giving them the prior expecation: <br>
$$
\tilde{u}^k_{i,j}\left( F_{i}\right) = \frac{1}{F_{i}}\sum_{f=1}^{F_{i}} \left(v_{j} + \epsilon^k_{f,j}\right), \; \epsilon^k_{f,j}\sim \mathcal{N}(0,\sigma^2)
$$
For ease of notation consider that each graduate have $F_{i}=i$ friends in each career. <br>

For $K$ times do the following: <br>
1. For each person $i$ draw $J\cdot F_i$ values of $\epsilon_{f,j}^{k}$, and calculate the prior expected utility of each career track, $\tilde{u}^k_{i,j}\left( F_{i}\right)$. <br>
Also draw their own $J$ noise terms, $\epsilon_{i,j}^k$
1. Each person $i$ chooses the career track with the highest expected utility: $$j_i^{k*}= \arg\max_{j\in{1,2\dots,J}}\left\{ \tilde{u}^k_{i,j}\left( F_{i}\right)\right\} $$
1. Store the chosen careers: $j_i^{k*}$, the prior expectation of the value of their chosen career: $\tilde{u}^k_{i,j=j_i^{k*}}\left( F_{i}\right)$, and the realized value of their chosen career track: $u^k_{i,j=j_i^{k*}}=v_{j=j_i^{k*}}+\epsilon_{i,j=j_i^{k*}}^k$.

In [67]:
u_s = []

for i in range(N):
    eps_fjk = np.random.normal(0, sigma, size=J * F[i])
    eps_ijk = np.random.normal(0, sigma, size=K)
    u_bar = 1/(i+1)*(v+eps_fjk.sum())

    u1, u2, u3 = u_bar

    u_s.append({
        'u1': u1,
        'u2': u2,
        'u3': u3
    })

tab4 = pd.DataFrame(u_s)
tab4

,u1,u2,u3
0,1.688437,2.688437,3.688437
1,-3.448593,-2.948593,-2.448593
2,2.919418,3.252751,3.586084
3,1.295610,1.545610,1.795610
4,1.530175,1.730175,1.930175
5,1.017616,1.184283,1.350950
6,1.080900,1.223757,1.366614
7,2.433986,2.558986,2.683986
8,0.770162,0.881273,0.992384
9,-0.602023,-0.502023,-0.402023


In [70]:
tab4.max()

u1    2.919418
u2    3.252751
u3    3.688437
dtype: float64

Chosen values will be: <br>
$i\in\left\{1,2\dots,N\right\}, N=10$ <br>
$F_i = i$<br>
So there are 10 graduates. The first has 1 friend in each career, the second has 2 friends, ... the tenth has 10 friends.

**Question 2:** Simulate and visualize: For each type of graduate, $i$, the share of graduates choosing each career, the average subjective expected utility of the graduates, and the average ex post realized utility given their choice. <br>
That is, calculate and visualize: <br>
$$
\begin{align*}
    \frac{1}{K} \sum_{k=1}^{K} \mathbb{I}\left\{ j=j_i^{k*} \right\}  \;\forall j\in\left\{1,2,\dots,J\right\}
\end{align*}
$$
$$
\begin{align*}
    \frac{1}{K} \sum_{k=1}^{K} \tilde{u}^k_{ij=j_i^{k*}}\left( F_{i}\right)
\end{align*}
$$
And 
$$
\begin{align*}
    \frac{1}{K} \sum_{k=1}^{K} u^k_{ij=j_i^{k*}} 
\end{align*}
$$
For each graduate $i$.

In [ ]:
# Write your answer here 

After a year of working in their career, the graduates learn $u^k_{ij}$ for their chosen job $j_i^{k*}$ perfectly. <br>
The can switch to one of the two remaining careers, for which they have the same prior as before, but it will now include a switching cost of $c$ which is known.
Their new priors can be written as: 
$$
\tilde{u}^{k,2}_{ij}\left( F_{i}\right) = \begin{cases}
            \tilde{u}^k_{ij}\left( F_{i}\right)-c & \text{if } j \neq j_i^{k*} \\
            u_{ij=j_i^{k*}} & \text{if } j = j_i^{k*}
        \end{cases}
$$

We will set $c=1$.

Their realized utility will be: <br>
$$
u^{k,2}_{ij}= \begin{cases}
            u_{ij}^k -c & \text{if } j \neq j_i^{k*} \\
            u_{ij=j_i^{k*}} & \text{if } j = j_i^{k*}
        \end{cases}
$$

**Question 3:** Following the same approach as in question 2, find the new optimal career choice for each $i$, $k$. Then for each $i$, calculate the average subjective expected utility from their new optimal career choice, and the ex post realized utility of that career. Also, for each $i$, calculate the share of graduates that chooses to switch careers, conditional on which career they chose in the first year. <br>

In [ ]:
# write your answer here

## 3. <a id='toc3_'></a>[Problem 3: Barycentric interpolation](#toc0_)

**Problem:** We have a set of random points in the unit square,

$$
\mathcal{X} = \{(x_1,x_2)\,|\,x_1\sim\mathcal{U}(0,1),x_2\sim\mathcal{U}(0,1)\}.
$$

For these points, we know the value of some function $f(x_1,x_2)$,

$$
\mathcal{F} = \{f(x_1,x_2) \,|\, (x_1,x_2) \in \mathcal{X}\}.
$$

Now we want to approximate the value $f(y_1,y_2)$ for some  $y=(y_1,y_2)$, where $y_1\sim\mathcal{U}(0,1)$ and $y_2\sim\mathcal{U}(0,1)$.

**Building block I**

For an arbitrary triangle $ABC$ and a point $y$, define the so-called barycentric coordinates as:

$$
\begin{align*}
  r^{ABC}_1 &= \frac{(B_2-C_2)(y_1-C_1) + (C_1-B_1)(y_2-C_2)}{(B_2-C_2)(A_1-C_1) + (C_1-B_1)(A_2-C_2)} \\
  r^{ABC}_2 &= \frac{(C_2-A_2)(y_1-C_1) + (A_1-C_1)(y_2-C_2)}{(B_2-C_2)(A_1-C_1) + (C_1-B_1)(A_2-C_2)} \\
  r^{ABC}_3 &= 1 - r_1 - r_2.
\end{align*}
$$

If $r^{ABC}_1 \in [0,1]$, $r^{ABC}_2 \in [0,1]$, and $r^{ABC}_3 \in [0,1]$, then the point is inside the triangle.

We always have $y = r^{ABC}_1 A + r^{ABC}_2 B + r^{ABC}_3 C$.

**Building block II**

Define the following points:

$$
\begin{align*}
A&=\arg\min_{(x_{1},x_{2})\in\mathcal{X}}\sqrt{\left(x_{1}-y_{1}\right)^{2}+\left(x_{2}-y_{2}\right)^{2}}\text{ s.t. }x_{1}>y_{1}\text{ and }x_{2}>y_{2}\\
B&=\arg\min_{(x_{1},x_{2})\in\mathcal{X}}\sqrt{\left(x_{1}-y_{1}\right)^{2}+\left(x_{2}-y_{2}\right)^{2}}\text{ s.t. }x_{1}>y_{1}\text{ and }x_{2}<y_{2}\\
C&=\arg\min_{(x_{1},x_{2})\in\mathcal{X}}\sqrt{\left(x_{1}-y_{1}\right)^{2}+\left(x_{2}-y_{2}\right)^{2}}\text{ s.t. }x_{1}<y_{1}\text{ and }x_{2}<y_{2}\\
D&=\arg\min_{(x_{1},x_{2})\in\mathcal{X}}\sqrt{\left(x_{1}-y_{1}\right)^{2}+\left(x_{2}-y_{2}\right)^{2}}\text{ s.t. }x_{1}<y_{1}\text{ and }x_{2}>y_{2}.
\end{align*}
$$

**Algorithm:**

1. Compute $A$, $B$, $C$, and $D$. If not possible return `NaN`.
1. If $y$ is inside the triangle $ABC$ return $r^{ABC}_1 f(A) + r^{ABC}_2 f(B) + r^{ABC}_3 f(C)$.
1. If $y$ is inside the triangle $CDA$ return $r^{CDA}_1 f(C) + r^{CDA}_2 f(D) + r^{CDA}_3 f(A)$.
1. Return `NaN`.



**Sample:**

In [ ]:
rng = np.random.default_rng(2024)

X = rng.uniform(size=(50,2))
y = rng.uniform(size=(2,))


**Questions 1:** Find $A$, $B$, $C$ and $D$. Illustrate these together with $X$, $y$ and the triangles $ABC$ and $CDA$.

In [ ]:
# write your answer here

**Question 2:** Compute the barycentric coordinates of the point $y$ with respect to the triangles $ABC$ and $CDA$. Which triangle is $y$ located inside?

In [ ]:
# write your answer here

Now consider the function:
$$
f(x_1,x_2) = x_1 \cdot x_2
$$

In [ ]:
f = lambda x: x[0]*x[1]
F = np.array([f(x) for x in X])

**Question 3:** Compute the approximation of $f(y)$ using the full algorithm. Compare with the true value.

In [ ]:
# write your answer here

**Question 4:** Repeat question 3 for all points in the set $Y$.

In [ ]:
Y = [(0.2,0.2),(0.8,0.2),(0.8,0.8),(0.8,0.2),(0.5,0.5)]

In [ ]:
# write your answer here